In [ ]:
import numpy as np
import matplotlib.pyplot as plt

a = np.linspace(0, 1, 100)
b = np.linspace(1, 2, 1000)
c = np.linspace(2, 3, 100)

# Agregar ruido a la distribución
ruido_a = np.random.normal(0, 0.01, a.shape)
ruido_b = np.random.normal(0, 0.01, b.shape)
ruido_c = np.random.normal(0, 0.01, c.shape)

a += ruido_a
b += ruido_b
c += ruido_c

distribucion = np.concatenate((a, b, c))
del a, b, c

plt.figure()
plt.hist(distribucion, bins=100)
plt.show()

counts, edges = np.histogram(distribucion, bins=100)

counts = np.insert(counts, 0, 0)
derivada_counts = np.diff(counts)


plt.figure()
plt.bar(edges[:-1], derivada_counts, width=np.diff(edges))
plt.show()

In [ ]:
import numpy as np
from scipy.signal import savgol_filter, find_peaks
import matplotlib.pyplot as plt

# Suponiendo que 'distribucion' es tu array de datos
# Calcular la derivada (diferencia entre puntos)
plt.figure()
plt.plot(distribucion)
plt.show()

gradiente = np.gradient(distribucion)

plt.figure()
plt.plot(gradiente)
plt.show()

# Suavizar la derivada para reducir el impacto del ruido
gradiente_suave = savgol_filter(gradiente, window_length=51, polyorder=3)

# Definir un umbral para detectar picos: por ejemplo, tomando un percentil alto
umbral = np.percentile(np.abs(gradiente_suave), 95)

# Detectar picos en la magnitud del gradiente
picos, _ = find_peaks(np.abs(gradiente_suave), height=umbral)

# Graficar para visualizar
plt.figure(figsize=(10, 5))
plt.plot(distribucion, label="Distribución")
plt.plot(picos, distribucion[picos], "ro", label="Cambios bruscos")
plt.legend()
plt.xlabel("Índice")
plt.ylabel("Valor")
plt.title("Detección de cambios abruptos en la distribución")
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import find_peaks

# Supongamos que 'data' es tu array con las posiciones (o el valor de interés) de las partículas
# Por ejemplo, podrías tener:
data = np.concatenate([
    np.random.normal(loc=0.5, scale=0.1, size=10000),  # zona de alta densidad (cerca del tubo)
    np.random.normal(loc=2.0, scale=0.1, size=5000)      # zona de baja densidad (fuera del tubo)
])

data = distribucion

# Crear histograma grueso
num_bins_coarse = 40  # cantidad de bins para la visión global
counts, bin_edges = np.histogram(data, bins=num_bins_coarse)
bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2

# Graficar histograma grueso
plt.figure(figsize=(8,4))
plt.bar(bin_centers, counts, width=(bin_edges[1]-bin_edges[0]), alpha=0.7)
plt.title("Histograma Grueso")
plt.xlabel("Posición")
plt.ylabel("Número de cuentas")
plt.show()

# Calcular la diferencia entre bins para detectar cambios bruscos
diff_counts = np.diff(counts)

plt.figure()
plt.plot(diff_counts)
plt.show()

# Usar find_peaks para identificar picos en la variación (cambiar el umbral según tu necesidad)
umbral = np.percentile(np.abs(diff_counts), 90)
picos, _ = find_peaks(np.abs(diff_counts), height=umbral)

print("Picos detectados (índices de bin):", picos)
# Aproximar la posición de la interfaz usando los bordes del histograma
interface_aproximadas = bin_edges[picos]
print("Posiciones aproximadas de la interfaz:", interface_aproximadas)


In [ ]:
import numpy as np
import pandas as pd
import kdsource as kds

# Defino la direccion de los archivos a comparar
original_file = "segundo_semestre/sources/source1/surface_source.h5"
synthetic_file = "/home/lucas/Documents/Proyecto_Integrador/PI/segundo_semestre/3-19-25/prueba.mcpl.gz"

# Cargo los archivos en DFs
# Cargar el archivo original en un DataFrame de pandas
SurfaceSourceFile = kds.SurfaceSourceFile(original_file, domain={"w": [0, 2]})
df_original = SurfaceSourceFile.get_pandas_dataframe()
# Seleccionar solo las columnas relevantes
df_original = df_original[["x", "y", "ln(E0/E)", "mu", "phi", "wgt"]]
# Liberar memoria eliminando el objeto SurfaceSourceFile
del SurfaceSourceFile

# Cargar el archivo sintético en un DataFrame de pandas
SurfaceSourceFile = kds.SurfaceSourceFile(synthetic_file, domain={"w": [0, 2]})
df_synthetic = SurfaceSourceFile.get_pandas_dataframe()
# Seleccionar solo las columnas relevantes
df_synthetic = df_synthetic[["x", "y", "ln(E0/E)", "mu", "phi", "wgt"]]
# Liberar memoria eliminando el objeto SurfaceSourceFile
del SurfaceSourceFile

del original_file, synthetic_file



In [ ]:
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

plt.rcParams.update(
    {
        "text.usetex": True,  # Usa LaTeX para el texto
        "font.family": "serif",  # Fuente tipo serif (similar a LaTeX)
        "font.serif": ["Computer Modern Roman"],  # Usa Computer Modern
        "axes.labelsize": 20,  # Tamaño de etiquetas de ejes
        "font.size": 20,  # Tamaño general de fuente
        "legend.fontsize": 18,  # Tamaño de fuente en la leyenda
        "xtick.labelsize": 18,  # Tamaño de fuente en los ticks de x
        "ytick.labelsize": 18,  # Tamaño de fuente en los ticks de y
        "font.weight": "bold",  # Fuente en negrita
        "axes.titleweight": "bold",  # Títulos de ejes en negrita
    }
)
bins = 1000

hist_lethargy_original, edges_lethargy_original = np.histogram(df_original["ln(E0/E)"], bins=bins, density=True)
hist_x_original, edges_x_original = np.histogram(df_original["x"], bins=bins, density=True)
hist_y_original, edges_y_original = np.histogram(df_original["y"], bins=bins, density=True)
hist_mu_original, edges_mu_original = np.histogram(df_original["mu"], bins=bins, density=True)
hist_phi_original, edges_phi_original = np.histogram(df_original["phi"], bins=bins, density=True)

hist_lethargy_synthetic, _ = np.histogram(df_synthetic["ln(E0/E)"], bins=edges_lethargy_original, density=True)
hist_x_synthetic, _ = np.histogram(df_synthetic["x"], bins=edges_x_original, density=True)
hist_y_synthetic, _ = np.histogram(df_synthetic["y"], bins=edges_y_original, density=True)
hist_mu_synthetic, _ = np.histogram(df_synthetic["mu"], bins=edges_mu_original, density=True)
hist_phi_synthetic, _ = np.histogram(df_synthetic["phi"], bins=edges_phi_original, density=True)

# del df_original, df_synthetic


In [ ]:

def plot_comparison(hist_original, hist_synthetic, edges, variable, folder=None):
    """
    Esta función genera una comparación gráfica entre dos histogramas (original y sintético) para una variable dada.
    
    Parámetros:
    - hist_original: numpy.ndarray
        Histograma de la variable original.
    - hist_synthetic: numpy.ndarray
        Histograma de la variable sintética.
    - edges: numpy.ndarray
        Bordes de los bins del histograma.
    - variable: str
        Nombre de la variable que se está comparando.
    - folder: str, opcional
        Carpeta donde se guardará la figura generada. Por defecto es una cadena vacía, lo que significa que la figura se guardará en el directorio actual.
    
    La función genera una figura con dos subgráficos:
    1. El primer subgráfico muestra los histogramas original y sintético en escala logarítmica.
    2. El segundo subgráfico muestra el error relativo entre los histogramas original y sintético.
    
    La figura se guarda como un archivo PNG con un nombre basado en la variable comparada.
    """
    fig = plt.figure(figsize=(16 * 1.25, 9 * 0.8))
    gs = gridspec.GridSpec(2, 1, height_ratios=[2.5, 1])  # La primera fila es el doble de alta que la segunda

    axs = [fig.add_subplot(gs[i]) for i in range(2)]  # Crear una lista de ejes

    plt.subplots_adjust(
        left=0.09, right=0.97, top=0.95, bottom=0.05, wspace=0.4, hspace=0.25
    )

    # Graficar hist_original y hist_synthetic en el primer gráfico de fig
    axs[0].plot((edges[:-1] + edges[1:]) / 2, hist_original, label="Original", color="blue", linestyle="--")
    axs[0].plot((edges[:-1] + edges[1:]) / 2, hist_synthetic, label="Sintético", color="red", linestyle="-")
    axs[0].set_yscale("log")
    axs[0].set_xlabel(variable)
    axs[0].set_ylabel("Frecuencia")
    axs[0].legend()
    axs[0].set_title(f"Distribución de {variable}")

    # Graficar error relativo en el segundo gráfico de fig
    axs[1].plot((edges[:-1] + edges[1:]) / 2, 100 * (hist_original - hist_synthetic) / hist_original, color="black")
    axs[1].set_ylabel("Error relativo (\%)")
    axs[1].axhline(0, color='gray', linestyle='--', linewidth=1)

    # Guardar la figura
    filename = f"comparacion_{variable}.png"
    if folder is not None:
        filename = folder + filename
    fig.savefig(filename)
    plt.close(fig)


In [ ]:
# Generar y guardar las figuras de comparación
plot_comparison(
    hist_lethargy_original,
    hist_lethargy_synthetic,
    edges_lethargy_original,
    "letargia",
    "/home/lucas/Documents/Proyecto_Integrador/PI/segundo_semestre/3-19-25/",
)
plot_comparison(
    hist_x_original,
    hist_x_synthetic,
    edges_x_original,
    "x",
    "/home/lucas/Documents/Proyecto_Integrador/PI/segundo_semestre/3-19-25/",
)
plot_comparison(
    hist_y_original,
    hist_y_synthetic,
    edges_y_original,
    "y",
    "/home/lucas/Documents/Proyecto_Integrador/PI/segundo_semestre/3-19-25/",
)
plot_comparison(
    hist_mu_original,
    hist_mu_synthetic,
    edges_mu_original,
    "mu",
    "/home/lucas/Documents/Proyecto_Integrador/PI/segundo_semestre/3-19-25/",
)
plot_comparison(
    hist_phi_original,
    hist_phi_synthetic,
    edges_phi_original,
    "phi",
    "/home/lucas/Documents/Proyecto_Integrador/PI/segundo_semestre/3-19-25/",
)

In [3]:
import kdsource.histograms as kdh

geometria = [True, 15, 15, 100, 3, 3]
z0 = 30
fuente_original = ["monoenergetica", "colimada"]

N_original = int(1e6)

columns_order = ["ln(E0/E)", "x", "y", "mu", "phi"]
micro_bins = [500] * len(columns_order)
# micro_bins = [50000, 40000, 35000,35000,30000]
macro_bins = [12, 12, 12, 12]
type = "equal_bins"
used_defined_edges = [
    [2.995732273553991 + 1e-9],
    # None,
    [-1.5, 1.5],
    [-1.5, 1.5],
    [1 - 1e-9],
    # None,
    None,
]

surface = kdh.SurfaceTrackProcessor(
    "/home/lucas/Documents/Proyecto_Integrador/PI/segundo_semestre/sources/source1/surface_source.h5",
    int(4.5e8),
)

surface.load_data()
surface.configure_binning(
    columns_order,
    micro_bins,
    macro_bins,
    type,
    used_defined_edges,
)
surface.load_simulation_info(geometria, z0, fuente_original)
surface.generate_tree()
surface.save_to_xml("/home/lucas/Documents/Proyecto_Integrador/PI/segundo_semestre/3-19-25/source_macros.xml")




Number of particles in file /home/lucas/Documents/Proyecto_Integrador/PI/segundo_semestre/sources/source1/surface_source.h5: 19180706
100% n, 0% g, 0% e-, 0% e+, 0% p, 


In [4]:
!kdtool resample -o /home/lucas/Documents/Proyecto_Integrador/PI/segundo_semestre/3-19-25/prueba2.mcpl -n 20000000 -m 2 /home/lucas/Documents/Proyecto_Integrador/PI/segundo_semestre/3-19-25/source_macros.xml


Geometria: 1.000000, 15.000000, 15.000000, 100.000000, 3.000000, 3.000000
z0: 30.000000
N_original: 450000000
fuente_original: monoenergetica, colimada
columns_order: ln(E0/E), x, y, mu, phi
micro_bins: 500, 500, 500, 500, 500
macro_bins: 12, 12, 12, 12
binning_type: equal_bins
used_defined_edges: [2.995732274553991], [-1.5, 1.5], [-1.5, 1.5], [0.999999999], None
factor_normalizacion: 0.041631
Particula 100000
Particula 200000
Particula 300000
Particula 400000
Particula 500000
Particula 600000
Particula 700000
Particula 800000
Particula 900000
Particula 1000000
Particula 1100000
Particula 1200000
Particula 1300000
Particula 1400000
Particula 1500000
Particula 1600000
Particula 1700000
Particula 1800000
Particula 1900000
Particula 2000000
Particula 2100000
Particula 2200000
Particula 2300000
Particula 2400000
Particula 2500000
Particula 2600000
Particula 2700000
Particula 2800000
Particula 2900000
Particula 3000000
Particula 3100000
Particula 3200000
Particula 3300000
Particula 3400000


In [ ]:
a = [2]
b = []
c = [3]

d = a + b + c
print(d)